In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movies = pd.read_csv("/content/drive/MyDrive/recomendacao/movies.csv")

In [ ]:
movies.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [ ]:
user_ratings = pd.read_csv("/content/drive/MyDrive/recomendacao/ratings.csv")

In [ ]:
user_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
def Convert(string):
      li = list(string.split("|"))
      return li 

In [ ]:
movies['genres'] = movies['genres'].apply(Convert)

In [ ]:
movies['year'] = movies.title.str.extract('(\(\d\d\d\d\))',expand=False)

In [ ]:
#Removendo os parenteses
movies['year'] = movies.year.str.extract('(\d\d\d\d)',expand=False)

In [ ]:
#Removendo o ano da coluna título
movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))', '')

<ipython-input-14-106093928401>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))', '')


In [ ]:
#Assegurando que removemos todos os espaços em branco da palavra
movies['title'] = movies['title'].apply(lambda x: x.strip())

In [ ]:
movies = movies.explode('genres')

In [ ]:
movies.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure,1995
0,1,Toy Story,Animation,1995
0,1,Toy Story,Children,1995
0,1,Toy Story,Comedy,1995
0,1,Toy Story,Fantasy,1995


In [ ]:
movies[movies["title"]=="Toy Story"]

,movieId,title,genres,year
0,1,Toy Story,Adventure,1995
0,1,Toy Story,Animation,1995
0,1,Toy Story,Children,1995
0,1,Toy Story,Comedy,1995
0,1,Toy Story,Fantasy,1995


In [ ]:
movies.genres.value_counts()

Drama                 4361
Comedy                3756
Thriller              1894
Action                1828
Romance               1596
Adventure             1263
Crime                 1199
Sci-Fi                 980
Horror                 978
Fantasy                779
Children               664
Animation              611
Mystery                573
Documentary            440
War                    382
Musical                334
Western                167
IMAX                   158
Film-Noir               87
(no genres listed)      34
Name: genres, dtype: int64

In [ ]:
movies[movies["genres"] == "(no genres listed)" ]

In [ ]:
#Mudando um dos generos
movies['genres'] = movies['genres'].replace(['(no genres listed)'], 'Others')

In [ ]:
#salvando dataframe
movies_fix = movies.to_csv("/content/drive/MyDrive/recomendacao/movies_fix.csv", index=True)

In [ ]:
# Iremos criar vetores one-hot encoding que irão vetores para cada genero
movies_emb = pd.get_dummies(movies, columns = ['genres'])

In [ ]:
movies_emb.head()

,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Horror,IMAX,Musical,Mystery,Others,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,Toy Story,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,Toy Story,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,Toy Story,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,Toy Story,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
movies_emb.drop("year", axis=1, inplace=True)

In [ ]:
movies_emb.columns = movies_emb.columns.str.replace("genres_", "")

In [ ]:
movies_emb.to_csv('/content/drive/MyDrive/recomendacao/embedings.csv')

In [ ]:
df2 = movies_emb.groupby(['movieId', 'title'], as_index=False).sum()

In [ ]:
df2.head()

,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Horror,IMAX,Musical,Mystery,Others,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale,0,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df2.shape

(9742, 22)

## Usando CrossTab

In [ ]:
movie_cross_table = pd.crosstab(movies['title'], movies['genres'])

In [ ]:
movie_cross_table.head()

genres,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Others,Romance,Sci-Fi,Thriller,War,Western
title,,,,,,,,,,,,,,,,,,,,
'71,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0
'Hellboy': The Seeds of Creation,1,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
'Round Midnight,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
'Salem's Lot,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0
'Til There Was You,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
toy_story_genres_ct = movie_cross_table[movie_cross_table.index == 'Black Butler: Book of the Atlantic']

In [ ]:
toy_story_genres_ct

genres,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Others,Romance,Sci-Fi,Thriller,War,Western
title,,,,,,,,,,,,,,,,,,,,
Black Butler: Book of the Atlantic,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


##Similaridade de Jaccard

In [ ]:
# Import numpy and the distance metric
from sklearn.metrics import jaccard_score

In [ ]:
# Extract just the rows containing GoldenEye and Toy Story
Black_Butler_values = movie_cross_table.loc['Black Butler: Book of the Atlantic'].values
toy_story_values = movie_cross_table.loc['Toy Story'].values

In [ ]:
Black_Butler_values

array([1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
toy_story_values

array([0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
print(jaccard_score(Black_Butler_values, toy_story_values))

0.5


## Comparando Todos os Filmes

In [ ]:
# Importando função 
from scipy.spatial.distance import pdist, squareform

In [ ]:
#Calcula a distância par a par
jaccard_distances = pdist(movie_cross_table.values, metric='jaccard')

In [ ]:
squareform(jaccard_distances)

array([[0.        , 0.875     , 0.8       , ..., 0.6       , 1.        ,
        1.        ],
       [0.875     , 0.        , 1.        , ..., 0.85714286, 0.83333333,
        0.83333333],
       [0.8       , 1.        , 0.        , ..., 1.        , 1.        ,
        0.66666667],
       ...,
       [0.6       , 0.85714286, 1.        , ..., 0.        , 1.        ,
        1.        ],
       [1.        , 0.83333333, 1.        , ..., 1.        , 0.        ,
        0.66666667],
       [1.        , 0.83333333, 0.66666667, ..., 1.        , 0.66666667,
        0.        ]])

In [ ]:
# Converter as distâncias para uma matriz quadrada
jaccard_similarity_array =  1 - squareform(jaccard_distances)

In [ ]:
jaccard_similarity_array

array([[1.        , 0.125     , 0.2       , ..., 0.4       , 0.        ,
        0.        ],
       [0.125     , 1.        , 0.        , ..., 0.14285714, 0.16666667,
        0.16666667],
       [0.2       , 0.        , 1.        , ..., 0.        , 0.        ,
        0.33333333],
       ...,
       [0.4       , 0.14285714, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.16666667, 0.        , ..., 0.        , 1.        ,
        0.33333333],
       [0.        , 0.16666667, 0.33333333, ..., 0.        , 0.33333333,
        1.        ]])

In [ ]:
# Wrap the array in a pandas DataFrame
jaccard_similarity_df = pd.DataFrame(jaccard_similarity_array, index=movie_cross_table.index, columns=movie_cross_table.index)

In [ ]:
jaccard_similarity_df.head()

title,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,*batteries not included,...,Zulu,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
title,,,,,,,,,,,,,,,,,,,,,
'71,1.000000,0.125,0.200000,0.333333,0.200000,0.0,0.0,0.25,0.166667,0.000000,...,0.600000,0.40,0.2,0.2,0.200000,0.400000,0.400000,0.400000,0.000000,0.000000
'Hellboy': The Seeds of Creation,0.125000,1.000,0.000000,0.000000,0.000000,0.0,0.2,0.00,0.142857,0.285714,...,0.111111,0.00,0.0,0.0,0.000000,0.142857,0.142857,0.142857,0.166667,0.166667
'Round Midnight,0.200000,0.000,1.000000,0.200000,0.333333,0.0,0.0,0.50,0.250000,0.000000,...,0.000000,0.25,0.0,0.0,0.333333,0.000000,0.000000,0.000000,0.000000,0.333333
'Salem's Lot,0.333333,0.000,0.200000,1.000000,0.200000,0.0,0.0,0.25,0.166667,0.000000,...,0.142857,0.75,0.5,0.5,0.200000,0.166667,0.166667,0.166667,0.000000,0.000000
'Til There Was You,0.200000,0.000,0.333333,0.200000,1.000000,0.5,0.0,0.50,0.666667,0.000000,...,0.000000,0.25,0.0,0.0,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000


## Fazendo Recomendações com Jaccard

In [ ]:
# Find the values for the movie Thor
jaccard_similarity_series = jaccard_similarity_df.loc['Toy Story']

In [ ]:
# Sort these values from highest to lowest
ordered_similarities = jaccard_similarity_series.sort_values(ascending=False)

In [ ]:
ordered_similarities

title
Wild, The                  1.0
Shrek the Third            1.0
Tale of Despereaux, The    1.0
The Good Dinosaur          1.0
Antz                       1.0
                          ... 
Monument Ave.              0.0
Competition, The           0.0
Conan the Barbarian        0.0
Monterey Pop               0.0
'71                        0.0
Name: Toy Story, Length: 9461, dtype: float64